In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Kirill/Projects/Test task_VK_2

/content/drive/MyDrive/Kirill/Projects/Test task_VK_2


# Объединение train test выборок с признаками

In [ ]:
import numpy as np
import pandas as pd
import torch

In [ ]:
train_df = pd.read_csv('data_preprocessing/train.csv')
train_df = train_df.drop('id', axis=1)

In [ ]:
test_df = pd.read_csv('data_preprocessing/test.csv')
test_df = test_df.drop('id', axis=1)

In [ ]:
train_df.head()

,lat,lon,score
0,56.228300,43.945535,0.080523
1,56.834244,53.141543,0.104424
2,45.042299,41.990170,0.067615
3,59.849408,30.387762,0.088038
4,59.839643,30.304308,0.099686


In [ ]:
test_df.head()

,lat,lon
0,55.782276,49.148234
1,56.841500,35.853059
2,55.356780,86.156697
3,56.755087,60.703962
4,59.907438,30.250297


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3084 entries, 0 to 3083
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lat     3084 non-null   float64
 1   lon     3084 non-null   float64
 2   score   3084 non-null   float64
dtypes: float64(3)
memory usage: 72.4 KB


In [ ]:
train_df.describe()

,lat,lon,score
count,3084.000000,3084.000000,3084.000000
mean,55.218066,43.942187,0.134855
std,3.832649,12.074348,0.118598
min,42.938677,20.472413,0.000000
25%,54.706904,37.512346,0.077965
50%,55.788525,39.754105,0.095098
75%,56.886170,49.191428,0.118312
max,61.286672,132.024317,0.945558


In [ ]:
features_df = pd.read_csv('data_preprocessing/features.csv')
features_df.head()

,lat,lon,0,1,2,3,4,5,6,7,...,353,354,355,356,357,358,359,360,361,362
0,59.837100,30.262701,0.411159,0.360032,0.350883,0.476389,0.373542,0.399907,0.397579,0.486442,...,0.041906,0.045916,0.085368,0.093044,0.014491,0.004091,0.043260,0.045916,0.060025,0.071592
1,59.935705,30.274201,0.500628,0.458489,0.522933,0.538892,0.482807,0.497257,0.541007,0.482956,...,0.041262,0.054171,0.066940,0.057510,0.007990,0.017897,0.042617,0.054171,0.047068,0.044251
2,59.929714,30.419863,0.450037,0.413593,0.445300,0.490658,0.403224,0.448099,0.516285,0.526028,...,0.044056,0.052290,0.126482,0.116896,0.006771,0.007329,0.045406,0.052290,0.088934,0.089945
3,56.111227,40.356119,0.160964,0.146153,0.301632,0.196871,0.157927,0.130723,0.246422,0.385300,...,0.043072,0.051181,0.042962,0.030555,0.010293,0.002216,0.044425,0.051181,0.030208,0.023510
4,56.233351,43.872728,0.102522,0.073835,0.270109,0.129343,0.106780,0.082463,0.201381,0.372377,...,0.047839,0.048137,0.135698,0.163684,0.009615,0.003579,0.049185,0.048137,0.095414,0.125946


In [ ]:
train_df.head()

,lat,lon,score
0,56.228300,43.945535,0.080523
1,56.834244,53.141543,0.104424
2,45.042299,41.990170,0.067615
3,59.849408,30.387762,0.088038
4,59.839643,30.304308,0.099686


In [ ]:
new_train_df = pd.DataFrame()
new_train_df.head()

""


In [ ]:
from tqdm import tqdm
from geopy.distance import geodesic as GD

for index_tr, row_tr in tqdm(train_df.iterrows()):
  min_distance = 50000
  index_min_distance = 0
  for index_f, row_f in features_df.iterrows():
    distance = GD((row_tr['lat'], row_tr['lon']), (row_f['lat'], row_f['lon']))
    if distance < min_distance:
      index_min_distance = index_f
      min_distance = distance

  new_row = row_tr[["lat", "lon"]].to_list() + features_df.loc[index_min_distance, ~features_df.columns.isin(['lat', 'lon'])].values.tolist() + [row_tr['score']] + [min_distance.km]
  new_row = pd.DataFrame(
    np.array(new_row).reshape(1, 367),
    columns=features_df.columns.tolist() + ['score', 'distance']
)
  new_train_df = new_train_df.append(new_row, ignore_index=True)


new_train_df.to_csv('data_preprocessing/train_with_features.csv')


'\nfor index_tr, row_tr in tqdm(train_df.iterrows()):\n  min_distance = 40000\n  index_min_distance = 0\n  for index_f, row_f in features_df.iterrows():\n    distance = GD((row_tr[\'lat\'], row_tr[\'lon\']), (row_f[\'lat\'], row_f[\'lon\']))\n    if distance < min_distance:\n      index_min_distance = index_f\n      min_distance = distance\n\n  new_row = row_tr[["lat", "lon"]].to_list() + features_df.loc[index_min_distance, ~features_df.columns.isin([\'lat\', \'lon\'])].values.tolist() + [row_tr[\'score\']] + [min_distance.km]\n  new_row = pd.DataFrame(\n    np.array(new_row).reshape(1, 367),\n    columns=features_df.columns.tolist() + [\'score\', \'distance\']\n)\n  new_train_df = new_train_df.append(new_row, ignore_index=True)\n\n\nnew_train_df.to_csv(\'out.csv\')\n'

In [ ]:
new_test_df = pd.DataFrame()

for index_tr, row_tr in tqdm(test_df.iterrows()):
  min_distance = 50000
  index_min_distance = 0
  for index_f, row_f in features_df.iterrows():
    distance = GD((row_tr['lat'], row_tr['lon']), (row_f['lat'], row_f['lon']))
    if distance < min_distance:
      index_min_distance = index_f
      min_distance = distance

  new_row = row_tr[["lat", "lon"]].to_list() + features_df.loc[index_min_distance, ~features_df.columns.isin(['lat', 'lon'])].values.tolist() + [min_distance.km]
  new_row = pd.DataFrame(
    np.array(new_row).reshape(1, 366),
    columns=features_df.columns.tolist() + ['distance']
)
  new_test_df = new_test_df.append(new_row, ignore_index=True)


new_test_df.to_csv('data_preprocessing/test_with_features.csv')


0it [00:00, ?it/s]<ipython-input-110-48e0b9ca004d>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_test_df = new_test_df.append(new_row, ignore_index=True)
1it [00:02,  2.17s/it]<ipython-input-110-48e0b9ca004d>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_test_df = new_test_df.append(new_row, ignore_index=True)
2it [00:03,  1.64s/it]<ipython-input-110-48e0b9ca004d>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_test_df = new_test_df.append(new_row, ignore_index=True)
3it [00:04,  1.49s/it]<ipython-input-110-48e0b9ca004d>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_test_df = new_test_df.append(new_row, ign

In [ ]:
train_f_df = pd.read_csv('data_preprocessing/train_with_features.csv')
train_f_df = train_f_df.drop(train_f_df.columns[0], axis= 1)

In [ ]:
train_f_df.head(10)

,lat,lon,0,1,2,3,4,5,6,7,...,355,356,357,358,359,360,361,362,score,distance
0,56.228300,43.945535,0.142355,0.144008,0.201290,0.145479,0.124842,0.135348,0.252250,0.291666,...,0.043170,0.037397,0.006636,0.006988,0.042022,0.046229,0.030355,0.028775,0.080523,0.070827
1,56.834244,53.141543,0.094093,0.096425,0.161355,0.089489,0.106608,0.085787,0.162016,0.437761,...,0.112922,0.118146,0.015303,0.005454,0.046132,0.050457,0.079399,0.090907,0.104424,0.046923
2,45.042299,41.990170,0.151723,0.148761,0.329598,0.159483,0.145691,0.140059,0.292674,0.388125,...,0.088930,0.103357,0.016793,0.004943,0.068319,0.069333,0.062530,0.079528,0.067615,0.036917
3,59.849408,30.387762,0.423663,0.289528,0.411117,0.516551,0.429777,0.407156,0.395440,0.570066,...,0.144839,0.146129,0.013272,0.005454,0.041311,0.041455,0.101842,0.112438,0.088038,0.063061
4,59.839643,30.304308,0.489140,0.361624,0.538862,0.546345,0.504863,0.499842,0.316289,0.527301,...,0.138435,0.126554,0.016522,0.010568,0.050615,0.059523,0.097339,0.097376,0.099686,0.012484
5,60.040549,30.392388,0.446181,0.355797,0.420148,0.514363,0.442338,0.420792,0.368047,0.628129,...,0.127291,0.126755,0.010834,0.006306,0.035630,0.046737,0.089503,0.097531,0.061768,0.045383
6,55.681707,37.580773,0.720869,0.754974,0.507914,0.743010,0.610044,0.749771,0.780490,0.468291,...,0.002824,0.002523,0.004334,0.001704,0.023291,0.031327,0.001986,0.001941,0.314885,0.005632
7,53.398583,58.979955,0.104391,0.075287,0.212984,0.114268,0.120666,0.086799,0.246274,0.378749,...,0.085270,0.096941,0.105769,0.076189,0.190297,0.206103,0.285259,0.331274,0.335736,0.051583
8,57.099150,65.569697,0.215478,0.203350,0.381153,0.214676,0.195422,0.201971,0.193623,0.342500,...,0.684451,0.690961,0.509751,0.125618,0.403168,0.398200,0.659939,0.685501,0.107926,0.037396
9,59.826303,30.327054,0.507335,0.476274,0.348698,0.525385,0.475351,0.508819,0.462240,0.536337,...,0.153317,0.150743,0.021939,0.012442,0.032876,0.061139,0.107803,0.115989,0.129243,0.036169


In [ ]:
train_f_df.describe()

,lat,lon,0,1,2,3,4,5,6,7,...,355,356,357,358,359,360,361,362,score,distance
count,3084.000000,3084.000000,3084.000000,3084.000000,3084.000000,3084.000000,3084.000000,3084.000000,3084.000000,3084.000000,...,3084.000000,3084.000000,3084.000000,3084.000000,3084.000000,3084.000000,3084.000000,3084.000000,3084.000000,3084.000000
mean,55.218066,43.942187,0.302841,0.275231,0.347248,0.337442,0.285094,0.288729,0.332240,0.383330,...,0.162978,0.162439,0.065615,0.038639,0.117849,0.123405,0.175405,0.190017,0.134855,0.093639
std,3.832649,12.074348,0.220498,0.197521,0.142191,0.261890,0.207133,0.215999,0.177084,0.110685,...,0.172815,0.171737,0.128162,0.080973,0.141534,0.139533,0.207862,0.222874,0.118598,0.276342
min,42.938677,20.472413,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001799
25%,54.706904,37.512346,0.127758,0.123571,0.245564,0.130945,0.120291,0.116502,0.196526,0.310674,...,0.063086,0.063771,0.006636,0.003409,0.038964,0.045571,0.049120,0.054110,0.077965,0.026486
50%,55.788525,39.754105,0.176640,0.183065,0.322754,0.184896,0.160698,0.168471,0.286670,0.381816,...,0.106119,0.106692,0.012730,0.006477,0.050012,0.055131,0.081249,0.088260,0.095098,0.043779
75%,56.886170,49.191428,0.466615,0.409549,0.434714,0.525355,0.455912,0.460693,0.459935,0.456955,...,0.156215,0.154573,0.042254,0.024288,0.129994,0.132430,0.215381,0.226916,0.118312,0.064822
max,61.286672,132.024317,1.000000,1.000000,1.000000,0.968861,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.996237,0.965907,0.945558,8.250536
